In [1]:
import polars as pl
import pandas as pd
import pyspark as ps

In [2]:
CSV_40M_PATH = 'data/Parking_Violations_40M.csv' # 9.65 GB
CSV_20M_PATH = 'data/Parking_Violations_20M.csv' # 4.89 GB
CSV_9M_PATH = 'data/Parking_Violations_9M.csv' # 1.95 GB
CSV_4M_PATH = 'data/Parking_Violations_4M.csv' # 855 MB
CSV_2M_PATH = 'data/Parking_Violations_2M.csv' # 428 MB
CSV_1M_PATH = 'data/Parking_Violations_1M.csv' # 214 MB
CSV_500K_PATH = 'data/Parking_Violations_500K.csv' # 107 MB

In [3]:
CSV_PATH = CSV_40M_PATH
OUT_JSON_NAME = 'CSV_40M_PATH'

In [4]:
comparison_results_dict = {
    'read_csv': {},
    'filter_one_condition': {},
    'filter_multiple_conditions': {},
    'create_new_column': {},
    'group_by': {}
}

# Let's compare

## 1. Read CSV

### 1.1. Polars

In [5]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
polars_df.collect()

10.4 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 10.4 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [6]:
comparison_results_dict['read_csv']['polars'] = _.best

### 1.2. Pandas

In [7]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')

KeyboardInterrupt: 

In [ ]:
comparison_results_dict['read_csv']['pandas'] = _.best

## 2. Filter rows based on one condition

### 2.1. Polars

In [ ]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.filter(pl.col(['Registration State']) == "NY")
result_polars_df.collect()

In [ ]:
comparison_results_dict['filter_one_condition']['polars'] = _.best

### 2.2. Pandas

In [ ]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
results_pandas_df = pandas_df.loc[pandas_df['Registration State'] == 'NY']

In [ ]:
comparison_results_dict['filter_one_condition']['pandas'] = _.best

## 3. Filter rows based on multiple conditions

### 3.1. Polars

In [ ]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.filter(
    (pl.col('Registration State') == 'NY') & \
    (pl.col('Summons Number') <= 1335096139) & \
    (pl.col('Summons Number') > 1335089433)
)
result_polars_df.collect()

In [ ]:
comparison_results_dict['filter_multiple_conditions']['polars'] = _.best

### 3.2. Pandas

In [ ]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
results_pandas_df = pandas_df.loc[
    (pandas_df['Registration State'] == 'NY') & \
    (pandas_df['Summons Number'] <= 1335096139) & \
    (pandas_df['Summons Number'] > 1335089433)
]

In [ ]:
comparison_results_dict['filter_multiple_conditions']['pandas'] = _.best

## 4. Create new column based on another

### 4.1. Polars

In [ ]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.with_columns([
    pl.col('Violation Code').map(lambda x: x + 10_000).alias('Clean Violation Code')
])
result_polars_df.collect()

In [ ]:
comparison_results_dict['create_new_column']['polars'] = _.best

### 4.2. Pandas

In [ ]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
pandas_df['Clean Violation Code'] = pandas_df['Violation Code'].apply(lambda x: x + 10_000)

In [ ]:
comparison_results_dict['create_new_column']['pandas'] = _.best

## 5. GroupBy

### 5.1. Polars

In [ ]:
%%timeit -o
polars_df = pl.scan_csv(CSV_PATH, ignore_errors=True)
result_polars_df = polars_df.groupby(by='Registration State').count()
result_polars_df.collect()

In [ ]:
comparison_results_dict['group_by']['polars'] = _.best

### 5.2. Pandas

In [ ]:
%%timeit -o
pandas_df = pd.read_csv(CSV_PATH, on_bad_lines='skip')
grouped_pandas_df = pandas_df.groupby(by='Registration State').count()

In [ ]:
comparison_results_dict['group_by']['pandas'] = _.best

## 6. RESULTS

In [ ]:
import json
print(json.dumps(comparison_results_dict, indent=4))

In [ ]:
with open(f'{OUT_JSON_NAME}_with_read.json', 'w') as file:
    json.dump(comparison_results_dict, file, indent=4)